# Tensorflow 2 Quickstart

## [1] import tensorflow dependency

In [1]:
import tensorflow as tf
# print("TensorFlow version:", tf.__version__)

/Users/kelcimensah/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## [2] load and prepare dataset
The MNIST dataset contains `60,000` 28x28 grayscale images of the ten (10) digits (0 through 9), along with a test dataset of `10,000` additional images.

The pixel values of the images range from `0` through `255`, we must scale these values to a range of `0` to `1`.

In [2]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0

## [3] build a machine learning model
The `Sequential` machine learning model is useful for stacking layers (functions with a known mathematical structure that can be reused and have trainable variables) where each layer has one input tensor (multidimensional array) and one output tensor.
* `Flatten` - reshapes the input data which expects 28x28 pixel images (like MNIST) and flattens them into a single-dimensional vector. This is necessary because the subsequent dense layers work with one-dimensional input.
* `Dense` - a fully connected (dense) layer with 128 neurons.
* `Dropout` - randomly sets 20% of the activations from the previous layer to zero during training.
* `Dense` = it has 10 neurons, corresponding to the 10 possible classes of digits (0-9).

The class with the highest probability would be the model's prediction.

In [6]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

predictions = model(x_train[:1]).numpy()
predictions

array([[-0.9007151 ,  0.2575215 ,  0.6284195 ,  0.7104968 , -0.20109035,
         0.49324715, -0.00767329, -0.78721285, -0.23085698,  0.58590573]],
      dtype=float32)

The `tf.nn.softmax()` function converts these logits (vector of raw (non-normalized) predictions that a classification model generates) to probabilities for each class:

In [7]:
tf.nn.softmax(predictions).numpy()

array([[0.03356833, 0.10689206, 0.15489036, 0.16813964, 0.06757294,
        0.13530685, 0.08199225, 0.03760305, 0.06559116, 0.1484434 ]],
      dtype=float32)

## [4] define a loss function for training
The loss function takes a vector of ground truth values and a vector of logits and returns a scalar loss for each example. Loss is zero if the model is sure of the correct class.

In [8]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to -tf.math.log(1/10) ~= 2.3
loss_fn(y_train[:1], predictions).numpy()

2.0002103

## [5] configure and compile model for training
* `optimizer="adam"` specifies the algorithm (adam or adaptive moment estimation) used for updating the model's weights during training.
* `loss=loss_fn` defines how the model measures the mismatch between its predictions and the true target values
* `metrics` defines a list of metrics the model should monitor during training and evaluation

In [9]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## [6] train model on data
The `model.fit()` function takes the training data (x_train), target labels (y_train), and tells model how many times to train designated dataset (epoch)

In [10]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 2s 894us/step - loss: 0.2992 - accuracy: 0.9133
Epoch 2/5
1875/1875 [==============================] - 2s 909us/step - loss: 0.1458 - accuracy: 0.9568
Epoch 3/5
1875/1875 [==============================] - 2s 946us/step - loss: 0.1087 - accuracy: 0.9675
Epoch 4/5
1875/1875 [==============================] - 2s 917us/step - loss: 0.0910 - accuracy: 0.9725
Epoch 5/5
1875/1875 [==============================] - 2s 888us/step - loss: 0.0757 - accuracy: 0.9761


## [7] evaluate accuracy of model

In [11]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0836 - accuracy: 0.9754 - 233ms/epoch - 743us/step


[0.08360092341899872, 0.9753999710083008]

## [8] 

In [12]:
# create a new Keras Sequential model named probability_model which reuses your existing trained model incorporating it as the first element in the new sequential model.
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

# feeds the first five samples of your testing data (x_test) into the new probability_model.
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[1.8084273e-10, 6.6520713e-11, 7.5041994e-06, 5.0142182e-05,
        2.3085863e-12, 1.7311338e-08, 1.3667572e-15, 9.9994099e-01,
        1.3102705e-07, 1.1291420e-06],
       [1.6978835e-09, 4.9913593e-04, 9.9933124e-01, 1.6824254e-04,
        1.1248403e-13, 2.3268149e-07, 4.7614779e-09, 2.2246097e-16,
        1.1387561e-06, 1.2531101e-11],
       [1.1412149e-07, 9.9968123e-01, 6.2087506e-06, 1.1492326e-05,
        1.2778924e-04, 2.2330196e-06, 3.9607748e-06, 1.3481063e-04,
        2.8727898e-05, 3.4879818e-06],
       [9.9964643e-01, 9.4672767e-09, 1.5518184e-04, 9.0586445e-07,
        3.8628168e-06, 6.1786377e-05, 1.2036127e-04, 7.9939791e-06,
        2.7989569e-07, 3.1950174e-06],
       [6.1903911e-06, 3.1167638e-10, 3.6445996e-04, 2.8368200e-08,
        9.9107307e-01, 4.4461349e-06, 3.6078904e-06, 1.9193423e-04,
        6.9596484e-07, 8.3555942e-03]], dtype=float32)>